# Chercher les fréquences des verbes défectifs dans Lexique3

In [1]:
import codecs,math,pickle,re,math
import pandas as pd
import numpy as np
import itertools as it
import matplotlib as mplt
import matplotlib.pyplot as plt

In [2]:
pd.__version__

u'0.23.1'

In [3]:
%matplotlib inline

In [4]:
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.set_palette("hls")
sns.set_color_codes("muted")
#sns.palplot(sns.color_palette("hls", 8))
#sns.palplot(sns.color_palette("husl", 8))

In [138]:
repHDR="/Users/gilles/ownCloud/Recherche/Boye/HDR/Memoire/figs/"
repDATA="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Longitudinales/"
nomRepertoire="/Volumes/gilles/Transfert/Copies-iMac-GB/Python/phonemisation/"
nomLexiqueOrg=nomRepertoire+"Lexique380-UTF8.txt"
Lexique3=pd.read_csv(nomLexiqueOrg,sep="\t")
nomVerbes3="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Samples/MGC-171229-Verbes3.pkl"

In [6]:
Lexique3.columns

Index([u'1_ortho', u'2_phon', u'3_lemme', u'4_cgram', u'5_genre', u'6_nombre',
       u'7_freqlemfilms2', u'8_freqlemlivres', u'9_freqfilms2',
       u'10_freqlivres', u'11_infover', u'12_nbhomogr', u'13_nbhomoph',
       u'14_islem', u'15_nblettres', u'16_nbphons', u'17_cvcv', u'18_p_cvcv',
       u'19_voisorth', u'20_voisphon', u'21_puorth', u'22_puphon', u'23_syll',
       u'24_nbsyll', u'25_cv-cv', u'26_orthrenv', u'27_phonrenv',
       u'28_orthosyll', u'29_cgramortho', u'30_deflem', u'31_defobs',
       u'32_old20', u'33_pld20', u'34_morphoder', u'35_nbmorph'],
      dtype='object')

Avec le changement de version de pandas, les anciens pickle ne sont plus compatibles avec la nouvelle version 0.23.1, pd.read_pickle permet de lire un fichier pkl contenant un DataFrame datant d'une version précédente de pandas mais pas un dictionnaire de DataFrame...

In [7]:
verbes3=pd.read_pickle(nomVerbes3)

In [13]:
verbesLex3=verbes3[(verbes3["freq"]>10000)]
paradigmes=verbesLex3.pivot_table(values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()


In [23]:
verbesLex3[verbesLex3["lexeme"]==u"clôturer"].sort_values("freq",ascending=False)

,ortho,phono,ext,cs,ms,vs,lexeme,freq,prob,case
72371,clôturer,klOtyre,"R""",V,,inf,clôturer,8.200005e+07,2.375222e-06,inf
72399,clôturé,klOtyre,NaN,K,MS,pp,clôturer,6.400003e+07,1.853832e-06,ppMS
72367,clôture,klOtyr,@,V,3S,pi,clôturer,6.100006e+07,1.766934e-06,pi3S
72355,clôturait,klOtyrE,"t""",V,3S,ii,clôturer,2.000003e+07,5.793230e-07,ii3S
72352,clôturaient,klOtyrE,"t""",V,3P,ii,clôturer,2.000001e+07,5.793223e-07,ii3P
72351,clôturai,klOtyrE,NaN,V,1S,ai,clôturer,2.000000e+07,5.793222e-07,ai1S
72400,clôturée,klOtyre,NaN,K,FS,pp,clôturer,1.000001e+07,2.896613e-07,ppFS
72350,clôtura,klOtyra,NaN,V,3S,ai,clôturer,9.000013e+06,2.606954e-07,ai3S
72369,clôturent,klOtyr,"@t""",V,3P,pi,clôturer,7.000009e+06,2.027630e-07,pi3P
72372,clôturera,klOtyr6ra,NaN,V,3S,fi,clôturer,2.000003e+06,5.793230e-08,fi3S


In [171]:
ipaIn = listerUnicode(u"SZNêôârEHO")
ipaOut= [u"ʃ",u"ʒ",u"ŋ",u"ɛ̃",u"ɔ̃",u"ɑ̃",u"ʁ",u"ɛ",u"ɥ",u"ɔ"]
toipa = dict(zip(ipaIn, ipaOut))

S Z N ê ô â r E H O


In [163]:
import unicodedata
def listerUnicode(chaine):
    result=[]
    comb=False
    prec=u""
    for char in chaine:
        print char,
        if unicodedata.combining(char):
            result.append(prec+char)
            prec=""
        else:
            result.append(prec)
            prec=char
    result.append(prec)
    return [r for r in result if r!=""]

def coderIPA(chaine,table=toipa):
    result=chaine
    for k in table:
        result=result.replace(k,table[k])
    return result

In [120]:
print coderIPA(u"êklyrE")

ɛ̃klyʁE


In [151]:
def makeTabularParadigme(lexeme,lDF,dictColours,title="",coulLim=False, cat="V"):
    row=lDF[lDF["lexeme"]==lexeme]
    tabular=[]
    def makeValue(case):
        if all(row[case].notnull()):
            return coderIPA(row[case].values[0])
        else:
            return "---"
    def makeLine6(tenseCode):
        line=[tabTemps[tenseCode]]
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            case=tenseCode+person
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
        return r" & ".join(line)+r"\\"

    def makeLine3(tenseCode):
        line=[tabTemps[tenseCode]]
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            if person in ["2S","1P","2P"]:
                case=tenseCode+person
                if case in dictColours:
                    line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
                else:
                    line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
#                line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
            else:
                line.append(r"---")
        return r" & ".join(line)+r"\\"
    
    def makeLineNF():
        line=["non-fini"]
        for case in ["inf","pP","ppMS","ppMP","ppFS","ppFP"]:
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
#            line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
        return r" & ".join(line)+r"\\"

    def makeLineMF(nombre):
        line=[]
        for genre in "mf":
            case=genre+nombre
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
        return r" & ".join(line)+r"\\"
    
    def makeLineCoulLim():
        line=[]
        for numLimite,limite in enumerate(listLimites):
            line.append(r"\cellcolor{%s}%s"%(listLimCoul[numLimite],"$<$"+str(limite)))
        return r"\hline\hline "+r" & ".join(line)+r"\\"
        
    if cat=="V":
        top=[
            r"\begin{center}",
            r"\begin{tabular}{ccccccc}",
            r"\toprule",
            " & ".join([lexeme]+[p+n for p in "123" for n in ["sg","pl"]])+r"\\",
            r"\midrule"
            ]
        bottom=[
            r"\bottomrule",
            r"\end{tabular}\\",
            title,
            r"\end{center}",
            r"\bigskip",
            r""
            ]
        tabular.append("\n".join(top))
        for tenseCode in ["pi","ii","fi","pc", "ps","ai", "is"]:
            tabular.append(makeLine6(tenseCode))
        tabular.append(makeLine3("pI"))
        tabular.append(makeLineNF())
    elif cat=="A":
        top=[
            r"\begin{center}",
            r"\begin{tabular}{cc}",
            r"\hline"
            ]
        bottom=[
            r"\hline",
            r"\end{tabular}\\",
            title,
            r"\end{center}",
            r"\bigskip",
            r""
            ]
        tabular.append("\n".join(top))
        for number in "sp":
            tabular.append(makeLineMF(number))
    if coulLim:
        tabular.append(makeLineCoulLim())
    tabular.append("\n".join(bottom))
    return "\n".join(tabular)

In [47]:
cases=paradigmes.columns.tolist()
cases.remove("lexeme")
cases

[u'ai1P',
 u'ai1S',
 u'ai2P',
 u'ai2S',
 u'ai3P',
 u'ai3S',
 u'fi1P',
 u'fi1S',
 u'fi2P',
 u'fi2S',
 u'fi3P',
 u'fi3S',
 u'ii1P',
 u'ii1S',
 u'ii2P',
 u'ii2S',
 u'ii3P',
 u'ii3S',
 u'inf',
 u'is1P',
 u'is1S',
 u'is2P',
 u'is2S',
 u'is3P',
 u'is3S',
 u'pI1P',
 u'pI2P',
 u'pI2S',
 u'pP',
 u'pc1P',
 u'pc1S',
 u'pc2P',
 u'pc2S',
 u'pc3P',
 u'pc3S',
 u'pi1P',
 u'pi1S',
 u'pi2P',
 u'pi2S',
 u'pi3P',
 u'pi3S',
 u'ppFP',
 u'ppFS',
 u'ppMP',
 u'ppMS',
 u'ps1P',
 u'ps1S',
 u'ps2P',
 u'ps2S',
 u'ps3P',
 u'ps3S']

In [90]:
etColours=["orange",
           "brown!50",
           "brown",
           "blue!10",
           "yellow!50",
           "blue!25",
           "teal!50",
           "blue!50",
           "cyan!50",
           "lime",
           "pink",
           "magenta!50",
          ]
etCells=[
    ["pi1P","pi2P"]+["ii"+p+n for p in "123" for n in "SP"],
    ["pi3P"],
    ["pi"+p+"S" for p in "123"],
    ["pP"],
    ["pI2S"],
    ["pI1P","pI2P"],
    ["ps"+p+"S" for p in "123"]+["ps3P"],
    ["ps1P","ps2P"],
    ["inf"],
    ["fi"+p+n for p in "123" for n in "SP"]+["pc"+p+n for p in "123" for n in "SP"],
    ["ai"+p+n for p in "123" for n in "SP"]+["is"+p+n for p in "123" for n in "SP"],
    ["pp"+g+n for g in "MF" for n in "SP"],
        ]
tabTemps={
    "pi":u"ind. prs",
    "ii":u"ind. ipf",
    "ai":u"ind. ps",
    "fi":u"ind. fut",
    "ps":u"subj. prs",
    "is":u"subj. ipf",
    "pc":u"cond. prs",
    "pI":u"imper. prs",
    "inf":u"non-fini"
    }
dictEtColours={}
for nET,ET in enumerate(etCells):
    for c in ET:
        dictEtColours[c]=etColours[nET]
dictEtColours

{'ai1P': 'pink',
 'ai1S': 'pink',
 'ai2P': 'pink',
 'ai2S': 'pink',
 'ai3P': 'pink',
 'ai3S': 'pink',
 'fi1P': 'lime',
 'fi1S': 'lime',
 'fi2P': 'lime',
 'fi2S': 'lime',
 'fi3P': 'lime',
 'fi3S': 'lime',
 'ii1P': 'orange',
 'ii1S': 'orange',
 'ii2P': 'orange',
 'ii2S': 'orange',
 'ii3P': 'orange',
 'ii3S': 'orange',
 'inf': 'cyan!50',
 'is1P': 'pink',
 'is1S': 'pink',
 'is2P': 'pink',
 'is2S': 'pink',
 'is3P': 'pink',
 'is3S': 'pink',
 'pI1P': 'blue!25',
 'pI2P': 'blue!25',
 'pI2S': 'yellow!50',
 'pP': 'blue!10',
 'pc1P': 'lime',
 'pc1S': 'lime',
 'pc2P': 'lime',
 'pc2S': 'lime',
 'pc3P': 'lime',
 'pc3S': 'lime',
 'pi1P': 'orange',
 'pi1S': 'brown',
 'pi2P': 'orange',
 'pi2S': 'brown',
 'pi3P': 'brown!50',
 'pi3S': 'brown',
 'ppFP': 'magenta!50',
 'ppFS': 'magenta!50',
 'ppMP': 'magenta!50',
 'ppMS': 'magenta!50',
 'ps1P': 'blue!50',
 'ps1S': 'teal!50',
 'ps2P': 'blue!50',
 'ps2S': 'teal!50',
 'ps3P': 'teal!50',
 'ps3S': 'teal!50'}

In [152]:
print makeTabularParadigme(u"bruiter",paradigmes,dictEtColours)

\begin{center}
\begin{tabular}{ccccccc}
\toprule
bruiter & 1sg & 1pl & 2sg & 2pl & 3sg & 3pl\\
\midrule
ind. prs & \cellcolor{brown}--- & \cellcolor{brown}--- & \cellcolor{brown}--- & \cellcolor{orange}--- & \cellcolor{orange}--- & \cellcolor{brown!50}---\\
ind. ipf & \cellcolor{orange}--- & \cellcolor{orange}--- & \cellcolor{orange}--- & \cellcolor{orange}--- & \cellcolor{orange}--- & \cellcolor{orange}---\\
ind. fut & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}---\\
cond. prs & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}---\\
subj. prs & \cellcolor{teal!50}--- & \cellcolor{teal!50}--- & \cellcolor{teal!50}--- & \cellcolor{blue!50}--- & \cellcolor{blue!50}--- & \cellcolor{teal!50}---\\
ind. ps & \cellcolor{pink}--- & \cellcolor{pink}--- & \cellcolor{pink}bʁɥita & \cellcolor{pink}--- & \cellcolor{pink}--- & \cellcolor{pink

In [136]:
testCase="pi3S"
testFin="o"
testParadigmes=paradigmes[(paradigmes[testCase].notnull())&(paradigmes[testCase].str.endswith(testFin))]
testListe=testParadigmes["lexeme"].tolist()
testParadigmes

case,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
895,clore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,kloz,kloz,klo,klo,NaN,kloz,NaN,NaN,NaN,kloz
2302,falloir,NaN,NaN,NaN,NaN,NaN,faly,NaN,NaN,NaN,...,NaN,NaN,NaN,faly,NaN,NaN,NaN,NaN,NaN,NaN
3713,prévaloir,NaN,NaN,NaN,NaN,NaN,prEvaly,NaN,NaN,NaN,...,NaN,NaN,NaN,prEvaly,NaN,NaN,NaN,NaN,NaN,NaN
5050,valoir,NaN,NaN,NaN,NaN,valyr,valy,vOdrô,vOdrE,vOdre,...,valy,valy,NaN,valy,NaN,vaj,NaN,NaN,vaj,vaj
5326,équivaloir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Ekivaly,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
for lexeme in testListe:
    print makeTabularParadigme(lexeme,dictEtColours)
    

\begin{center}
\begin{tabular}{ccccccc}
\toprule
clore & 1sg & 1pl & 2sg & 2pl & 3sg & 3pl\\
\midrule
ind. prs & \cellcolor{brown}klo & \cellcolor{brown}--- & \cellcolor{brown}klo & \cellcolor{orange}--- & \cellcolor{orange}--- & \cellcolor{brown!50}kloz\\
ind. ipf & \cellcolor{orange}--- & \cellcolor{orange}--- & \cellcolor{orange}--- & \cellcolor{orange}--- & \cellcolor{orange}--- & \cellcolor{orange}---\\
ind. fut & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}klOʁa & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}---\\
cond. prs & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}---\\
subj. prs & \cellcolor{teal!50}kloz & \cellcolor{teal!50}--- & \cellcolor{teal!50}kloz & \cellcolor{blue!50}--- & \cellcolor{blue!50}--- & \cellcolor{teal!50}---\\
ind. ps & \cellcolor{pink}--- & \cellcolor{pink}--- & \cellcolor{pink}--- & \cellcolor{pink}--- & \cellcolor{pink}--- & \cellcolor{pink

In [132]:
paradigmes[paradigmes["lexeme"].str.endswith("ore")]

case,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
895,clore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,kloz,kloz,klo,klo,NaN,kloz,NaN,NaN,NaN,kloz
1449,déclore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,dEkloz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018,enclore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,âkloz,âkloz,âklo,âklo,NaN,NaN,NaN,NaN,NaN,NaN
5200,éclore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ekloz


In [169]:
nomLex=repDATA+"Longitudinal-50-T600000-F33424-X-paradigmes.csv"
dfLex=pd.read_csv(nomLex,sep=";",index_col=0,encoding="utf8")
nomSwim2=repDATA+"Longitudinal-50-T600000-F33424-X-paradigmes-Swim2.csv"
dfSwim2=pd.read_csv(nomSwim2,sep=";",index_col=0,encoding="utf8")

In [170]:
lexeme=u"clore"
print makeTabularParadigme(lexeme,dfLex,dictEtColours)
print makeTabularParadigme(lexeme,dfSwim2,dictEtColours)

\begin{center}
\begin{tabular}{ccccccc}
\toprule
clore & 1sg & 1pl & 2sg & 2pl & 3sg & 3pl\\
\midrule
ind. prs & \cellcolor{brown}klo & \cellcolor{brown}--- & \cellcolor{brown}klo & \cellcolor{orange}--- & \cellcolor{orange}--- & \cellcolor{brown!50}---\\
ind. ipf & \cellcolor{orange}--- & \cellcolor{orange}--- & \cellcolor{orange}--- & \cellcolor{orange}--- & \cellcolor{orange}--- & \cellcolor{orange}---\\
ind. fut & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}---\\
cond. prs & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}--- & \cellcolor{lime}---\\
subj. prs & \cellcolor{teal!50}--- & \cellcolor{teal!50}--- & \cellcolor{teal!50}--- & \cellcolor{blue!50}--- & \cellcolor{blue!50}--- & \cellcolor{teal!50}---\\
ind. ps & \cellcolor{pink}--- & \cellcolor{pink}--- & \cellcolor{pink}--- & \cellcolor{pink}--- & \cellcolor{pink}--- & \cellcolor{pink}---\